# Running ALS on MovieLens (pySpark)

[ALS](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/recommendation.html#ALS) (Alternating Least Squares) is a well-known collaborative filtering algorithm.

This notebook provides an example of how to utilize and evaluate ALS pySpark ML (DataFrame-based API) implementation, meant for large-scale distributed datasets. We use a smaller dataset in this example to run ALS efficiently on Data Science Virtual Machine.

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import numpy as np
from zipfile import ZipFile
import papermill as pm
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType

from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation


print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))


System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Spark version: 2.3.1


Set the default parameters.

In [2]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 0. Set up Spark context

The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap. 

In [3]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = SparkSession \
    .builder \
    .appName("ALS pySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.cores", "32")\
    .config("spark.executor.memory", "8g")\
    .config("spark.memory.fraction", "0.9")\
    .config("spark.memory.stageFraction", "0.3")\
    .config("spark.executor.instances", 1)\
    .config("spark.executor.heartbeatInterval", "36000s")\
    .config("spark.network.timeout", "10000000s")\
    .config("spark.driver.maxResultSize", "50g")\
    .getOrCreate()


### 1. Download the MovieLens dataset

In [5]:
# MovieLens data have different data-format for each size of dataset
data_header = False
if MOVIELENS_DATA_SIZE == "100k":
    separator = "\t"
    data_name = "u.data"
    data_folder = "ml-100k"
elif MOVIELENS_DATA_SIZE == "1m":
    separator = "::"
    data_name = "ratings.dat"
    data_folder = "ml-1m"
elif MOVIELENS_DATA_SIZE == "10m":
    separator = "::"
    data_name = "ratings.dat"
    data_folder = "ml-10M100K"
elif MOVIELENS_DATA_SIZE == "20m":
    separator = ","
    data_name = "ratings.csv"
    data_folder = "ml-20m"
    data_header = True
else:
    raise ValueError("Invalid data size. Should be one of {100k, 1m, 10m, or 20m}")

# Download dataset zip file and decompress if haven't done yet
data_path = os.path.join(data_folder, data_name)
if not os.path.exists(data_path):
    filename = "ml-" + MOVIELENS_DATA_SIZE + ".zip"
    filepath = maybe_download(
        "http://files.grouplens.org/datasets/movielens/" + filename, filename
    )

    with ZipFile(filepath, "r") as zf:
        zf.extractall()

    # remove zip file we already used
    os.remove(filepath)


In [6]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField("UserId", IntegerType()),
        StructField("MovieId", IntegerType()),
        StructField("Rating", FloatType()),
        StructField("Timestamp", IntegerType()),
    )
)

# pySpark's read csv currently doesn't support multi-character delimiter, thus we manually handle that
if len(separator) > 1:
    raw_data = spark.sparkContext.textFile(data_path) \
        .map(lambda l: l.split(separator)).map(lambda s: ','.join(map(str, s)))
    data = spark.read.csv(raw_data, schema=schema, sep=',', header=data_header)
else:
    data = spark.read.csv(data_path, schema=schema, sep=separator, header=data_header)

data.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



### 2. Split the data using the Spark random splitter provided in utilities

In [7]:
train, test = spark_random_split(data, ratio=0.75, seed=123)
print ("N train", train.count())
print ("N test", test.count())

N train 75193
N test 24807


### 3. Train the ALS model on the training data, and get the top-k recommendations for our testing data

To predict movie ratings, we use the rating data in the training set as users' explicit feedbacks.

When our goal is to recommend top k movies a user is likely to watch, on the other hand, we utilize the ratings as implicit feedbacks.

In [36]:
header = {
    "userCol": "UserId",
    "itemCol": "MovieId",
    "ratingCol": "Rating",
}


# implicitPrefs=True for recommendation, False for rating prediction
als = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=True,
    alpha=0.1,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

In [37]:
model = als.fit(train)


In [38]:
recommendations = model.recommendForUserSubset(test, TOP_K)
recommendations.show()
    

+------+--------------------+
|UserId|     recommendations|
+------+--------------------+
|   471|[[418, 0.72847724...|
|   463|[[285, 1.1634982]...|
|   833|[[218, 1.2006702]...|
|   496|[[173, 0.8102586]...|
|   148|[[50, 0.8821161],...|
|   540|[[276, 0.85914886...|
|   392|[[286, 0.9766282]...|
|   243|[[275, 1.29936], ...|
|   623|[[222, 0.86506325...|
|   737|[[222, 0.5029861]...|
|   897|[[118, 1.1622006]...|
|   858|[[286, 0.95543224...|
|    31|[[302, 0.76913214...|
|   516|[[127, 0.5144711]...|
|   580|[[121, 0.8675087]...|
|   251|[[181, 1.2622311]...|
|   451|[[243, 1.4820322]...|
|    85|[[52, 1.0961343],...|
|   137|[[121, 0.8577174]...|
|   808|[[288, 1.0185469]...|
+------+--------------------+
only showing top 20 rows



In [39]:
# Convert to reco util's ranking evaluator format
top_k = recommendations.select('UserId', F.explode('recommendations').alias('r')) \
    .select('UserId', 'r.*')
top_k.show()

+------+-------+----------+
|UserId|MovieId|    rating|
+------+-------+----------+
|   471|    418|0.72847724|
|   471|    419|0.67094696|
|   471|     71| 0.6393287|
|   471|    588|0.63891846|
|   471|     99| 0.6329491|
|   471|    501|0.61551756|
|   471|    143| 0.6047367|
|   471|    151| 0.5938562|
|   471|     95| 0.5764764|
|   471|    465|0.54725754|
|   463|    285| 1.1634982|
|   463|     13| 1.1459694|
|   463|    124| 1.1449559|
|   463|    246| 1.0542792|
|   463|     15| 1.0238962|
|   463|    242| 0.9382459|
|   463|     93| 0.9239277|
|   463|    287| 0.9204046|
|   463|    111|0.90601474|
|   463|      1| 0.8508104|
+------+-------+----------+
only showing top 20 rows



### 4. Evaluate how well ALS performs

In [40]:
test.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|     1|      2|   3.0|876893171|
|     1|      3|   4.0|878542960|
|     1|      4|   3.0|876893119|
|     1|      9|   5.0|878543541|
|     1|     11|   2.0|875072262|
|     1|     17|   3.0|875073198|
|     1|     25|   4.0|875071805|
|     1|     28|   4.0|875072173|
|     1|     30|   3.0|878542515|
|     1|     33|   4.0|878542699|
|     1|     43|   4.0|878542869|
|     1|     48|   5.0|875072520|
|     1|     49|   3.0|878542478|
|     1|     52|   4.0|875072205|
|     1|     59|   5.0|876892817|
|     1|     62|   3.0|878542282|
|     1|     65|   4.0|875072125|
|     1|     66|   4.0|878543030|
|     1|     71|   3.0|876892425|
|     1|     78|   1.0|878543176|
+------+-------+------+---------+
only showing top 20 rows



In [41]:
rank_eval = SparkRankingEvaluation(test, top_k, k = TOP_K, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="rating", 
                                    relevancy_method="top_k")

In [42]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.025738
NDCG:	0.100178
Precision@K:	0.095966
Recall@K:	0.078518


### 5. Evaluate rating prediction

In [43]:
als_prediction = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

model_prediction = als_prediction.fit(train)

prediction = model_prediction.transform(test)
prediction.show()


+------+-------+------+---------+----------+
|UserId|MovieId|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|   406|    148|   3.0|879540276| 3.2573113|
|    27|    148|   3.0|891543129|  2.342981|
|   606|    148|   3.0|878150506| 4.1623316|
|   916|    148|   2.0|880843892| 3.0077415|
|   236|    148|   4.0|890117028| 4.6593585|
|   602|    148|   4.0|888638517| 3.4857998|
|   663|    148|   4.0|889492989|  4.033325|
|   372|    148|   5.0|876869915| 3.2888944|
|   190|    148|   4.0|891033742| 3.5253122|
|     1|    148|   2.0|875240799| 2.7576623|
|   297|    148|   3.0|875239619| 2.7936513|
|   178|    148|   4.0|882824325| 3.4072607|
|   308|    148|   3.0|887740788| 3.3479352|
|   923|    148|   4.0|880387474| 3.3072624|
|    54|    148|   3.0|880937490|  4.562116|
|   430|    148|   2.0|877226047| 2.6006272|
|    92|    148|   2.0|877383934| 2.5384903|
|   447|    148|   4.0|878854729|  3.989952|
|   374|    148|   4.0|880392992|  3.290884|
|   891|  

In [44]:
rating_eval = SparkRatingEvaluation(test, prediction, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="prediction")
print("Model:\tALS\nRMSE:\t%.2f" % rating_eval.rmse())

Model:	ALS
RMSE:	1.12


In [ ]:
# Record results with papermill for tests
pm.record("map", rank_eval.map_at_k())
pm.record("ndcg", rank_eval.ndcg_at_k())
pm.record("precision", rank_eval.precision_at_k())
pm.record("recall", rank_eval.recall_at_k())
